# 解释模型

可以使用 Azure 机器学习并通过解释器来解释模型，该解释器可以量化每个特征对预测标签的影响程度**。有许多常见的解释器，每种解释器适用于不同类型的建模算法；但其基本使用方法是相同的。

## 安装 SDK 包

除了最新版本的 azureml-sdk 和 azureml-widgets 包之外，还需要 azureml-explain-model 包才能在此笔记本中运行代码************。你还将使用 Azure ML 可解释性库 (azureml-interpret)****。可以使用此库解释许多典型的模型，即使还未在 Azure ML 试验中训练它们或还未在 Azure ML 工作区中注册它们。

运行下面的单元以验证这些包是否已安装。 

In [ ]:
!pip show azureml-explain-model azureml-interpret

## 解释模型

我们从在 Azure 机器学习之外训练的模型开始 - 运行以下单元格以训练决策树分类模型。

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 加载糖尿病数据集
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# 分隔特征和标签
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练决策树模型
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

训练过程中根据保留的验证数据集生成了一些模型评估指标，以便你了解其预测的准确性；但是数据中的特征如何影响预测呢？

### 获取模型解释器

让我们从你先前安装的 Azure ML 可解释性库中获得适合该模型的解释器。有多种类型的解释器。在此示例中，你将使用表格解释器，这是一种“黑盒”解释器，可用于通过调用适当的 [SHAP](https://github.com/slundberg/shap) 模型解释器来解释多种类型的模型**。

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# “特征”和“类”字段是可选的
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### 获取全局特征重要性**

首先要通过评估整体特征重要性来解释模型 - 换言之，根据整体训练数据集来量化每个特征对预测的影响程度**。

In [ ]:
# 可使用此处的训练数据或测试数据
global_tab_explanation = tab_explainer.explain_global(X_train)

# 获取按重要性划分的主要特征
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

已对特征重要性进行排名，最重要的特征排在最前面。

### 获取局部特征重要性**

现在你有了一个整体的了解，但是如何解释单个观察结果呢？现在为单个预测生成本地解释，量化每个特征对预测每个可能标签值的决策的影响程度**。在本例中，它是二进制模型，因此有两个可能的标签（非糖尿病和糖尿病）；你可以量化每个特征对数据集中单个观察结果的每个标签值的影响程度。只需评估测试数据集中的前两个案例。

In [ ]:
# 获取我们想要解释的观察结果（前两个）
X_explain = X_test[0:2]

# 获取预测
predictions = model.predict(X_explain)

# 获取本地解释
local_tab_explanation = tab_explainer.explain_local(X_explain)

# 获取每个可能的标签的特征名称和重要性
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

## 为模型训练试验增加可解释性

如你所见，你可为在 Azure 机器学习之外训练的模型生成解释，但当你在 Azure 机器学习工作区中使用试验来训练和注册模型时，可以生成模型解释并记录它们。

运行以下单元格中的代码以连接到你的工作区。

> 备注****：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 使用试验训练和解释模型

现在来创建试验并将其所需的文件放在本地文件夹中 - 在本例中，我们将仅使用相同的糖尿病数据 CSV 文件来训练模型。

In [ ]:
import os, shutil
from azureml.core import Experiment

# 为试验文件创建文件夹
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# 将数据文件复制到试验文件夹
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

现在，我们将创建一个训练脚本，它看起来与任何其他 Azure ML 训练脚本相似，但它包括以下功能：

- 导入我们之前使用的用于生成模型解释的相同库，并将这些库用于生成全局解释
- 使用 ExplanationClient 库将解释上传到试验输出****

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 导入 Azure ML 运行库
from azureml.core.run import Run

# 导入用于模型解释的库
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据集
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# 分隔特征和标签
X, y = data[features].values, data['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练决策树模型
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在 outputs 文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# 获取解释
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# 获取解释客户端并上传解释
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# 完成运行
run.complete()

现在可以运行试验了。请注意，训练环境中包含 azureml-interpret 库，以便脚本可以创建 TabularExplainer 和使用 ExplainerClient 类************。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 创建用于试验的 Python 环境
explain_env = Environment("explain-env")

# 创建一组包依赖项（包括 azureml-interpret 包）
packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
explain_env.python.conda_dependencies = packages

# 创建脚本配置
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env) 

# 提交试验
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

## 检索特征重要性值

试验运行完成后，可以使用 ExplanationClient 类从为运行注册的解释中检索特征重要性****。

In [ ]:
from azureml.interpret import ExplanationClient

# 获取特征解释
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 整体特征重要性
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## 在 Azure 机器学习工作室中查看模型解释

还可以单击“运行详细信息”小组件中的“查看运行详细信息”链接以在 Azure 机器学习工作室中查看运行，并查看“解释”选项卡********。然后：

1.选择表格解释器的解释 ID。
2.查看“聚合特征重要性”图表，其中显示整体全局特征重要性****。
3.查看“单个特征重要性”图表，其中显示测试数据中的各个数据点****。
4.选择单个点以查看所选数据点的单个预测的局部特征重要性。
5.“新队列”按钮用于使用以下设置来定义数据的子集****：
    - **数据集队列名称**：25 岁以下
    - **选择筛选器**：数据集
        - 年龄小于 25 岁（确保添加此筛选器再保存新队列）。
6.建立第二个新的队列，其名为“25 岁及以上”，并使用“年龄大于或等于 25 岁”的筛选器****。
6.查看“聚合特征重要性”可视化，并比较定义的两个队列的相对特征重要性****。通过比较队列，可以了解到特征如何影响数据总体的多个子集的不同预测结果。



详细信息****：有关在 Azure ML 中使用解释器的详细信息，请参阅此[文档](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability)。 